In [146]:
import numpy as np
import scipy.interpolate as si   

def bspline(cv, n=100, degree=3, periodic=False):
    """ Calculate n samples on a bspline

        cv :      Array ov control vertices
        n  :      Number of samples to return
        degree:   Curve degree
        periodic: True - Curve is closed
                  False - Curve is open
    """

    # If periodic, extend the point array by count+degree+1
    cv = np.asarray(cv)
    count = len(cv)

    if periodic:
        factor, fraction = divmod(count+degree+1, count)
        cv = np.concatenate((cv,) * factor + (cv[:fraction],))
        count = len(cv)
        degree = np.clip(degree,1,degree)

    # If opened, prevent degree from exceeding count-1
    else:
        degree = np.clip(degree,1,count-1)


    # Calculate knot vector
    kv = None
    if periodic:
        kv = np.arange(0-degree,count+degree+degree-1)
    else:
        kv = np.clip(np.arange(count+degree+1)-degree,0,count-degree)

    # Calculate query range
    u = np.linspace(periodic,(count-degree),n)


    # Calculate result
    return np.array(si.splev(u, (kv,cv.T,degree)))

In [156]:
import scipy.misc
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


random.seed(123)  # predictable pseudo random sequence

swirl_points= 255

def create_swirl(size):
    data = np.zeros((size, size))

    cv = np.array([
        [ random.random(),  random.random()],
        [ random.random(),  random.random()],
        [ random.random(),  random.random()],
        [ random.random(),  random.random()]])

    points= (bspline(cv, n=swirl_points, degree=2) * size).astype("int")

    for p in points: 
        data[p[0], p[1]] += 1/swirl_points
    
    return data

swirls= [create_swirl(32) for _ in range(256)]
np.save("swirls4x"+str(size)+".npy", swirls)  
        
